In [ ]:
import pandas as pd
from pathlib import Path
import shutil
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import numpy as np

In [ ]:
# === SETTINGS YOU CAN EDIT =====================================

xlsx_path = Path("Large_Model_Questions_WITH_BERTSCORE_PRF_debertaxlargemnli_Align.xlsx")
output_path = Path("Large_Model_Questions_WITH_BERTSCORE_PRF_debertaxlargemnli_Align_with_summary.xlsx")

ps_sheets = ["PS1", "PS2", "PS3", "PS4", "PS5"]

# Canonical column names we want in the summary
numeric_metrics = {
    "Avg Similarity": "Similarity",
    "Avg Grade level": "Grade level",
    "Avg Reading Ease": "Reading Ease",
    "1--2": "sim_1_2_f1",
    "2--3": "sim_2_3_f1",
    "3--4": "sim_3_4_f1",
    "4--5": "sim_4_5_f1",
    "5--6": "sim_5_6_f1",
}

bool_metrics = {
    "Bloom_Level_Check": "Assigned Level Check",
    "Grade_compliant": "Grade Level Check",
    "precision_compliant_rate": "precision_compliant",
    "recall_compliant_rate": "recall_compliant",
    "f1_compliant_rate": "f1_compliant",
}

ordered_cols = list(numeric_metrics.keys()) + list(bool_metrics.keys())

In [ ]:
# === HELPERS ====================================================

def find_column(df, target):
    """Case/space-insensitive column lookup."""
    target_clean = target.strip().lower()
    for col in df.columns:
        if col.strip().lower() == target_clean:
            return col
    return None


def normalize_columns(df):
    """Strip whitespace in column names."""
    df = df.copy()
    df.columns = [col.strip() for col in df.columns]
    return df

def summarize_sheet(df: pd.DataFrame) -> pd.DataFrame:
    """
    Per-model summary for one PS sheet.

    - Strips whitespace from column names.
    - Forward-fills Model and Prompt to handle merged cells from Excel.
    - Normalizes Model values (strip + upper) so names are consistent.
    - Drops rows that don't contain a real question/metrics.
    - Resolves metric columns case-insensitively.
    - Computes:
        * numeric means (Similarity, Grade level, Reading Ease, ...)
        * boolean TRUE rates for compliance columns.
    """
    df = normalize_columns(df)

    # ---- forward-fill & standardize Model column ----
    model_col = find_column(df, "Model")
    if model_col is None:
        raise KeyError(
            f"'Model' column not found in sheet. Columns present: {list(df.columns)}"
        )

    # ffill the model column to account for merged cells in Excel
    df[model_col] = df[model_col].ffill()

    # rename to canonical "Model" if needed
    if model_col != "Model":
        df = df.rename(columns={model_col: "Model"})

    # NORMALISE MODEL VALUES (this is the important new part)
    df["Model"] = (
        df["Model"]
        .astype(str)
        .str.strip()
        .str.upper()   # use .str.upper() so GPT4 == gpt4
    )

    # ---- forward-fill Prompt as well (merged cells) ----
    prompt_col = find_column(df, "Prompt")
    if prompt_col:
        df[prompt_col] = df[prompt_col].ffill()
        if prompt_col != "Prompt":
            df = df.rename(columns={prompt_col: "Prompt"})

    # ---- drop structural/blank rows (no real question / metrics) ----
    q_col = find_column(df, "Questions")
    if q_col:
        df = df[df[q_col].notna()]

    sim_col = find_column(df, "Similarity")
    if sim_col:
        df = df[df[sim_col].notna()]

    # ---- normalize boolean-like columns (TRUE/FALSE strings) ----
    for _, col in bool_metrics.items():
        actual = find_column(df, col)
        if actual:
            df[actual] = df[actual].replace(
                {"TRUE": True, "FALSE": False, "true": True, "false": False}
            )
            if actual != col:
                df = df.rename(columns={actual: col})

    # ---- resolve metric columns actually present ----
    resolved_numeric = {}
    for out_name, col in numeric_metrics.items():
        actual = find_column(df, col)
        if actual:
            resolved_numeric[out_name] = actual
        else:
            print(
                f"⚠ Warning: column '{col}' not found (case-insensitive) "
                f"→ skipping '{out_name}'"
            )

    resolved_boolean = {}
    for out_name, col in bool_metrics.items():
        actual = find_column(df, col)
        if actual:
            resolved_boolean[out_name] = actual
        else:
            print(
                f"⚠ Warning: column '{col}' not found (case-insensitive) "
                f"→ skipping '{out_name}'"
            )

    g = df.groupby("Model")
    agg_parts = {}

    # ---- numeric means ----
    for out_name, actual_col in resolved_numeric.items():
        agg_parts[out_name] = g[actual_col].mean()  # skipna=True by default

    # ---- boolean TRUE rate: #True / (#True + #False) ----
    for out_name, actual_col in resolved_boolean.items():
        true_count = g[actual_col].apply(lambda x: x.eq(True).sum())
        tf_count = g[actual_col].apply(lambda x: x.isin([True, False]).sum())
        agg_parts[out_name] = true_count / tf_count

    summary = pd.concat(agg_parts, axis=1)

    # ---- canonicalise index again & merge any accidental duplicates ----
    # (e.g., if some model names slipped through differently earlier)
    summary.index = (
        summary.index.to_series()
        .astype(str)
        .str.strip()
        .str.upper()
    )
    summary = summary.groupby(summary.index).mean()

    # ---- reorder columns consistently ----
    summary = summary[[c for c in ordered_cols if c in summary.columns]]

    return summary


def compute_cv(ps_summaries, ps_sheet_names):
    """
    Coefficient of variation across PS1..PS5 for each model & metric.
    CV = std / mean, computed over non-empty values, ignoring NaNs.
    """
    # union of models
    models = sorted(set().union(*[ps_summaries[s].index for s in ps_sheet_names]))
    # union of columns
    cols = sorted(set().union(*[ps_summaries[s].columns for s in ps_sheet_names]))

    cv_data = {col: [] for col in cols}

    for m in models:
        for col in cols:
            vals = []
            for s in ps_sheet_names:
                df = ps_summaries[s]
                if col in df.columns and m in df.index:
                    v = df.at[m, col]
                    if pd.notna(v):
                        vals.append(float(v))
            if len(vals) >= 2 and np.mean(vals) != 0:
                vals_series = pd.Series(vals, dtype=float)
                cv = vals_series.std(ddof=0) / vals_series.mean()
            else:
                cv = np.nan
            cv_data[col].append(cv)

    cv_df = pd.DataFrame(cv_data, index=models)
    # order columns like the others
    cv_df = cv_df[[c for c in ordered_cols if c in cv_df.columns]]
    return cv_df

In [ ]:
# Load all sheets
book = pd.read_excel(xlsx_path, sheet_name=None, engine="openpyxl")

ps_summaries = {}
all_rows = []

for sheet in ps_sheets:
    df = book[sheet]
    print(f"\nProcessing {sheet} — columns detected:\n{list(df.columns)}")
    summary = summarize_sheet(df)
    ps_summaries[sheet] = summary
    all_rows.append(df)

# TOTAL summary (mean over all rows that actually have each value)
total_df = pd.concat(all_rows, ignore_index=True)
ps_summaries["Total"] = summarize_sheet(total_df)

# CV summary across PS1..PS5 only
cv_summary = compute_cv(ps_summaries, ps_sheets)
ps_summaries["CV"] = cv_summary

# ---- rounding & percentage formatting ----
numeric_cols = set(numeric_metrics.keys())
bool_cols = set(bool_metrics.keys())

for ps_name, df in ps_summaries.items():
    df = df.copy()
    for col in df.columns:
        if ps_name == "CV":
            # CV as percentage for all metrics
            df[col] = (df[col] * 100).round(2)
        else:
            if col in numeric_cols:
                df[col] = df[col].round(2)
            elif col in bool_cols:
                # convert compliance rates to %
                df[col] = (df[col] * 100).round(2)
    ps_summaries[ps_name] = df

# ---- build Summary sheet blocks ----
blocks = []
for ps_name in ["PS1", "PS2", "PS3", "PS4", "PS5", "Total", "CV"]:
    summary = ps_summaries[ps_name]
    block = summary.copy()
    block.insert(0, "PS", ps_name)
    block = block.reset_index()  # Model becomes a column named 'index'
    block = block.rename(columns={"index": "Model"})
    blocks.append(block)
    # separator row
    blocks.append(pd.DataFrame([[None] * block.shape[1]], columns=block.columns))

summary_sheet = pd.concat(blocks, ignore_index=True)

# 1. Start from a copy of your original file (so we don't overwrite it in-place)
shutil.copy2(xlsx_path, output_path)

# 2. Open the copied workbook with openpyxl
wb = load_workbook(output_path)

# 3. If a "Summary" sheet already exists, remove it
if "Summary" in wb.sheetnames:
    std = wb["Summary"]
    wb.remove(std)

# 4. Create a fresh Summary sheet
ws = wb.create_sheet("Summary")

# 5. Dump the pandas DataFrame into the Summary sheet
for r_idx, row in enumerate(dataframe_to_rows(summary_sheet, index=False, header=True), start=1):
    ws.append(row)

# 6. Save the updated workbook
wb.save(output_path)

print(f"\nSummary (with Total + CV) written to: {output_path}")
